<a href="https://colab.research.google.com/github/andresum97/03MAIR_algoritmos_de_optimizacion/blob/main/Actividad_Guiada_3_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AG3 - Actividad Guiada 3

Nombre: Andrés Urízar

https://colab.research.google.com/drive/1Sylfi3efLc615xLhlNuGVtc01hfVNyxm?usp=sharing

https://github.com/andresum97/03MAIR_algoritmos_de_optimizacion


## Carga de librerias


In [1]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.8 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


In [24]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Generar valores aleatorios

In [25]:
#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos 

gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [26]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file) #Datos del fichero

#Nodos
nodos = list(problem.get_nodes()) #Tiene funcion definida para obtener nodos

#Aristas
Aristas = list(problem.get_edges()) #Tiene funcion definida para obtener aristas

In [27]:
# Funcion para probar

#Distancia entre nodos
problem.get_weight(0, 3)

23

## Declaración de funciones básicas

In [28]:
# Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
    return problem.get_weight(a,b)

# Genera una solucion aleatoria empezando en el nodo 0
def crear_solucion(nodos):
    solucion = [nodos[0]]
    #en cada paso se elige un elemento no elegido y eliminando el 0 y los que
    #hayamos elegido anteriormente
    #Si hay que quitar el nodo 0 porque si parece afectar la distancia
    for nodo in nodos[1:]:
      solucion += [random.choice(list(set(nodos) - set({nodos[0]}) - set(solucion)))]
    return solucion

# Calcula la distancia total de la trayectoria/solucion
def distancia_total(solucion, problem):
  dist = 0
  #recorrer todos los pasos y calcula distancia acumulando las aristas
  for i in range(len(solucion)-1):
    dist += distancia(solucion[i], solucion[i+1], problem)
  return dist + distancia(solucion[len(solucion)-1], solucion[0], problem)

solucion = crear_solucion(nodos)
distancia_total(solucion, problem)

print(solucion)

[0, 17, 4, 16, 31, 14, 3, 1, 26, 22, 6, 21, 7, 34, 38, 39, 11, 35, 27, 23, 12, 25, 20, 29, 8, 37, 13, 9, 36, 24, 10, 32, 5, 41, 28, 40, 33, 18, 19, 2, 30, 15]


## Búsqueda aleatoria

In [29]:
# Busqueda aleatoria

def busqueda_aleatoria(problem, n):
  nodos = list(problem.get_nodes())

  mejor_resultado = []
  #Se comienza con un valor alto para que el primer resultado encontrado
  #lo reemplace
  mejor_distancia = 999999999

  for i in range(n):
    resultado = crear_solucion(nodos)    #solucion aleatoria
    distancia = distancia_total(solucion, problem) #distancia de solucion aleatoria

    if distancia < mejor_distancia:
      mejor_resultado = resultado
      mejor_distancia = distancia

  return mejor_resultado, distancia

solucion, dist = busqueda_aleatoria(problem, 5000)
print(solucion)
print(dist)

[0, 12, 13, 7, 33, 25, 4, 34, 18, 37, 22, 35, 21, 40, 24, 9, 20, 17, 28, 19, 38, 32, 41, 14, 36, 16, 1, 10, 30, 27, 8, 6, 26, 23, 31, 3, 5, 11, 2, 15, 29, 39]
5006


## Búsqueda local

In [30]:
# Busqueda local
def genera_vecina(solucion):
  mejor_resultado = []
  mejor_distancia = 9999999999

  # Se intercambian nodos, se comienza por el primer nodo
  # y se utiliza el nodo siguiente al seleccionado hasta el final
  for i in range(1, len(solucion) - 1):
    for j in range(i+1, len(solucion)):
      # Se procede a intercambiar nodo i con nodo j
      vecino = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      # Calcula la nueva distancia de la solucion obtenida
      nueva_distancia = distancia_total(vecino, problem)

      # Si es mejor se realiza el intercambio
      if nueva_distancia <= mejor_distancia:
        mejor_distancia = nueva_distancia
        mejor_resultado = vecino

  return mejor_resultado

print(distancia_total(solucion, problem))
nueva_solucion = genera_vecina(solucion)
print(distancia_total(nueva_solucion, problem))

4671
4269


In [31]:
#Busqueda local

def busqueda_local(problem):
  mejor_resultado = []

  # Se genra una solucion aleatoria
  sol_referencia = crear_solucion(nodos)
  mejor_distancia = distancia_total(sol_referencia, problem)

  cont = 0
  # Se realiza un loop infinito hasta que se haga un retorno
  # (criterio de parada)
  while(True):
    vecino = genera_vecina(sol_referencia)
    distancia = distancia_total(vecino, problem)

    # Se comprueba que mejora el resultado
    if distancia < mejor_distancia:
      mejor_resultado = vecino
      mejor_distancia = distancia
    
    else:
      # Cuando ya no tiene a donde mas avanzar
      print('Distancia', mejor_distancia)
      return mejor_resultado

    sol_referencia = vecino


resultado = busqueda_local(problem)
print('Distancia ', resultado)

Distancia 1745
Distancia  [0, 1, 15, 16, 14, 6, 4, 8, 41, 23, 9, 40, 24, 21, 39, 22, 38, 29, 30, 32, 28, 27, 2, 3, 17, 35, 36, 37, 5, 10, 25, 11, 12, 18, 26, 13, 19, 7, 31, 20, 33, 34]


### Propuesta de mejora de búsqueda local

In [35]:
# Genera una solucion VNS reducida
def crear_solucion_2(nodos):
    solucion = []  # Se elige k <- 1
    distancia = 99999999

    cont = 0
    # Se repite hasta que k = kmax
    while cont <= len(nodos):
      # Se genera una solucion x'
      solucion = crear_solucion(nodos) 
      # Se vuelve a utilizar la funcion original para crear una solucion aleatoria
      # Si la solucion obtenida es mejor, entonces se reemplaza y se vuelve a probar
      dist_temp = distancia_total(solucion, problem)
      if dist_temp > distancia:
        distancia = dist_temp
        cont = 1
      else:
        # En caso no es una mejor solucion, se continua con el ciclo
        # solucion = [nodos[0]]
        cont += 1

    print('No. Iteraciones ', cont)
    
    return solucion

def busqueda_local_2(problem):
  mejor_resultado = []

  # Se genra una solucion aleatoria
  sol_referencia = crear_solucion_2(nodos)
  mejor_distancia = distancia_total(sol_referencia, problem)

  cont = 0
  # Se realiza un loop infinito hasta que se haga un retorno
  # (criterio de parada)
  while(True):
    vecino = genera_vecina(sol_referencia)
    distancia = distancia_total(vecino, problem)

    # Se comprueba que mejora el resultado
    if distancia < mejor_distancia:
      mejor_resultado = vecino
      mejor_distancia = distancia
    
    else:
      # Cuando ya no tiene a donde mas avanzar
      print('Distancia', mejor_distancia)
      return mejor_resultado

    sol_referencia = vecino

# Prueba 
resultado = busqueda_local_2(problem)
print('Resultado ', resultado)

No. Iteraciones  43
Distancia 1565
Resultado  [0, 26, 5, 13, 19, 14, 16, 15, 37, 17, 31, 36, 35, 20, 33, 34, 32, 30, 29, 10, 25, 8, 9, 23, 41, 11, 12, 18, 4, 27, 28, 38, 22, 24, 40, 21, 39, 2, 3, 6, 1, 7]


Como se puede observar arriba, hubo una mejora de 200 de distancia, indicando que el metodo de VNS Reducida, si pudo encontrar una mejor solucion a la búsqueda local con otros operadores de vecindad. Como aqui se trata a fuerza de encontrar una solucion mejor, no quedarse solamente con la primera, sino que ir probando distintas soluciones x' para encontrar la x que de la mejor distancia.

## Recocido simulado

In [10]:
# Recocido simulado o simulated annealing

# Es para generar aleatoriamente un x en el entorno de vecindad de la
# la solucion de referencia
# No se debe elegir tan aleatorio, sino algo random pero mas dirigido
# considerar eso para nodo a,b. Verificar quiza nodos alejados.
def genera_vecina_aleatorio(solucion):
  #Seleccion de nodos de manera aleatoria
  a,b = sorted(random.sample(range(1, len(solucion)), 2))

  return solucion[:a] + [solucion[b]] + solucion[a+1:b] + [solucion[a]] + solucion[b+1:]

In [11]:
# Maneja probabilidad de transparencia para aceptar peores soluciones
def probabilidad(temperatura, delta):
  return random.random() < math.exp(-1*delta / temperatura)

# Maneja el descenso de temperatura
def bajar_temperatura(temperatura):
  return temperatura*0.99

In [12]:
def recocido_simulado(problem, temperatura):
  sol_referencia = crear_solucion(nodos)
  distancia_referencia = distancia_total(sol_referencia, problem)

  solucion = []    # x* del algoritmo
  distancia = 9999999999   # F* del algoritmo

  n = 0
  while temperatura > 0.001:
    n += 1
    # Aqui se desarrolla la propuesta de mejora
    vecino = genera_vecina_aleatorio(sol_referencia)
    distancia_vecino = distancia_total(vecino, problem)

    # Si es mejor hace el cambio
    if distancia_vecino < distancia:
      solucion = vecino
      distancia = distancia_vecino

    # En caso no sea mejor, se realiza el salto por medio
    # de la temperatura, cambiando una solucion diferente
    if (
        distancia_vecino < distancia_referencia 
        or probabilidad(
            temperatura, 
            abs(distancia_referencia - distancia_vecino)
            )
        ):
      sol_referencia = vecino
      distancia_referencia = distancia_vecino

    # Se baja la temperatura
    temperatura = bajar_temperatura(temperatura)

  print("mejor distancia ", distancia)
  return solucion


sol = recocido_simulado(problem, 10000000)
print("Solucion ", sol)
    

mejor distancia  2130
Solucion  [0, 3, 1, 7, 17, 31, 20, 33, 34, 32, 30, 29, 10, 12, 11, 18, 15, 37, 16, 14, 5, 26, 2, 28, 23, 41, 25, 4, 36, 35, 27, 40, 21, 24, 38, 22, 39, 9, 8, 13, 19, 6]


### Oportunidad de mejora de recorrido simulado

In [80]:
def genera_vecina_aleatorio_2(solucion):
  #Seleccion de nodos de manera aleatoria, pero se intenta elegir elementos separados
  longitud_cuarto = len(solucion)//4
  len_ = 0
  # se trata que los elementos tengan un criterio de lejania, para expandir busqueda
  # entonces se busca que tenga una distancia de 1/4 del tamaño de la lista
  while len_ < longitud_cuarto:
    a,b = sorted(random.sample(range(1, len(solucion)), 2))
    len_ = solucion.index(b) -  solucion.index(a)

  return solucion[:a] + [solucion[b]] + solucion[a+1:b] + [solucion[a]] + solucion[b+1:]

def recocido_simulado(problem, temperatura):
  sol_referencia = crear_solucion(nodos)
  distancia_referencia = distancia_total(sol_referencia, problem)

  solucion = []    # x* del algoritmo
  distancia = 9999999999   # F* del algoritmo

  n = 0
  while temperatura > 0.001:
    n += 1
    # Aqui se desarrolla la propuesta de mejora
    vecino = genera_vecina_aleatorio_2(sol_referencia)
    distancia_vecino = distancia_total(vecino, problem)

    # Si es mejor hace el cambio
    if distancia_vecino < distancia:
      solucion = vecino
      distancia = distancia_vecino

    # En caso no sea mejor, se realiza el salto por medio
    # de la temperatura, cambiando una solucion diferente
    if (
        distancia_vecino < distancia_referencia 
        or probabilidad(
            temperatura, 
            abs(distancia_referencia - distancia_vecino)
            )
        ):
      sol_referencia = vecino
      distancia_referencia = distancia_vecino

    # Se baja la temperatura
    temperatura = bajar_temperatura(temperatura)

  print("mejor distancia ", distancia)
  return solucion


sol = recocido_simulado(problem, 10000000)
print("Solucion ", sol)
    

mejor distancia  2226
Solucion  [0, 37, 31, 20, 33, 34, 3, 6, 14, 16, 19, 5, 40, 21, 24, 39, 22, 38, 30, 32, 29, 28, 2, 27, 17, 15, 10, 23, 9, 8, 41, 12, 11, 25, 18, 4, 26, 13, 35, 36, 7, 1]


Se trato de elegir una elección no tan aleatoria. Se trato de que se buscaran elementos distintos, pero se asegura que son elementos algo alejados. En el caso del algoritmo de *genera_vecina_aleatorio_2* se trato de que hubiera una distancia de al menos 1/4 del tamaño de la lista entre ambos nodos seleccionados, pero se probo acortar o reducir distancias entre elementos, y se noto que mientras más largo la distancia entre ambos nodos, el resultado salia peor, el resultado más cercano a que se obtuvo a la respuesta del algoritmo totalmente aleatorio fue con 1/8, pero ahi se asegura que los elementos estan muy cerca, pero igual no hubo una mejora en el resultando, indicando que con una elección no tan aleatoria, no hubo mejoras.